# Udacity Data Engineering Capstone Project
## Project

Load, explore and clean the fact tables with Pandas.

**Download the data** 
* Use data_utils.py functions
* Find all the csv files in ../data/source folder


## Libraries

In [1]:
import os
import configparser
import pandas as pd
import numpy as np
from datetime import datetime
import glob

## ```flights``` table

In [2]:
input_data = "../data/source/flightlist_20201201_20201231.csv.gz"

df = pd.read_csv( input_data, compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894751 entries, 0 to 1894750
Data columns (total 16 columns):
 #   Column        Dtype  
---  ------        -----  
 0   callsign      object 
 1   number        object 
 2   icao24        object 
 3   registration  object 
 4   typecode      object 
 5   origin        object 
 6   destination   object 
 7   firstseen     object 
 8   lastseen      object 
 9   day           object 
 10  latitude_1    float64
 11  longitude_1   float64
 12  altitude_1    float64
 13  latitude_2    float64
 14  longitude_2   float64
 15  altitude_2    float64
dtypes: float64(6), object(10)
memory usage: 231.3+ MB


In [3]:
df.isna().sum()

callsign              0
number          1347027
icao24               15
registration     343346
typecode         737605
origin           491611
destination      370281
firstseen             0
lastseen              0
day                   0
latitude_1            0
longitude_1           0
altitude_1            0
latitude_2           48
longitude_2          48
altitude_2        10126
dtype: int64

In [4]:
df.head()

,callsign,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
0,ETH728,NaN,040188,NaN,NaN,KEWR,EBBR,2020-11-30 02:23:10+00:00,2020-12-01 05:42:23+00:00,2020-12-01 00:00:00+00:00,40.670083,-74.182809,0.0,50.894989,4.506102,30.48
1,TAM9560,NaN,e48df6,PT-MUI,B77W,SBGR,OTHH,2020-11-30 02:30:53+00:00,2020-12-01 04:58:15+00:00,2020-12-01 00:00:00+00:00,-23.417545,-46.426290,1219.2,25.213297,51.645329,236.22
2,CKS416,NaN,a96b14,N706CK,B744,KSUU,RKSG,2020-11-30 04:16:19+00:00,2020-12-01 04:57:26+00:00,2020-12-01 00:00:00+00:00,38.292023,-121.888110,304.8,36.922302,126.834352,1661.16
3,JST8992,NaN,7c6b0b,VH-VFH,A320,WSSL,YMML,2020-11-30 05:51:16+00:00,2020-12-01 05:54:57+00:00,2020-12-01 00:00:00+00:00,1.428223,103.875034,304.8,-37.663786,144.874201,281.94
4,JST8993,NaN,7c6b1c,VH-VFY,A320,WSSS,YMML,2020-11-30 06:11:13+00:00,2020-12-01 07:47:17+00:00,2020-12-01 00:00:00+00:00,1.316849,103.979823,304.8,-37.662643,144.854795,198.12


In [5]:
# process opensky data
df1 = df.copy()

# Remove rows where origin AND destination are unknown 
df1 = df1.dropna( how='all', subset=['origin', 'destination'])

col = 'destination'
df1.loc[ ~df1[col].astype(str).str.isalnum()]

,callsign,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
225517,OYHJY,NaN,45a159,NaN,NaN,EKMB,EK_4,2020-12-04 14:40:01+00:00,2020-12-04 14:53:31+00:00,2020-12-04 00:00:00+00:00,54.698973,11.451166,0.0,55.155697,11.249917,777.24
1599328,DOC86,NaN,47a1ce,NaN,NaN,EKRS,EK_4,2020-12-27 03:24:33+00:00,2020-12-27 03:38:29+00:00,2020-12-27 00:00:00+00:00,55.424722,11.806474,0.0,55.149399,11.588557,396.24


In [6]:
df1.loc[ df1.duplicated( subset=['callsign', 'icao24', 'origin', 'destination'], keep=False)].sort_values( by=['callsign', 'icao24', 'day'])

,callsign,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
576515,0,NaN,00893d,NaN,NaN,FAOR,NaN,2020-12-10 11:38:01+00:00,2020-12-10 11:50:42+00:00,2020-12-10 00:00:00+00:00,-26.081039,28.255281,1828.8,-26.982971,29.037383,9578.34
592083,0,NaN,00893d,NaN,NaN,FAOR,NaN,2020-12-10 16:13:07+00:00,2020-12-10 16:26:57+00:00,2020-12-10 00:00:00+00:00,-26.080536,28.255437,1828.8,-26.911697,29.492072,9829.80
645629,0,NaN,00893d,NaN,NaN,FAOR,NaN,2020-12-11 11:25:55+00:00,2020-12-11 11:39:59+00:00,2020-12-11 00:00:00+00:00,-26.080032,28.254763,1828.8,-27.202698,28.268496,10256.52
660675,0,NaN,00893d,NaN,NaN,FAOR,NaN,2020-12-11 16:06:45+00:00,2020-12-11 16:20:22+00:00,2020-12-11 00:00:00+00:00,-26.087219,28.255126,1828.8,-26.884598,29.417915,10050.78
258060,0,NaN,ada07e,N9773Y,NaN,3TX2,TE92,2020-12-04 23:13:35+00:00,2020-12-04 23:45:02+00:00,2020-12-04 00:00:00+00:00,33.076249,-97.358430,0.0,31.639709,-97.505062,2194.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972637,ZYU,NaN,7c8204,NaN,NaN,YWBN,YWBN,2020-12-16 23:51:12+00:00,2020-12-17 00:08:41+00:00,2020-12-17 00:00:00+00:00,-34.166840,150.800918,304.8,-34.167160,150.816221,510.54
1363725,ZYU,NaN,7c8204,NaN,NaN,YWBN,YWBN,2020-12-22 21:15:46+00:00,2020-12-22 21:51:22+00:00,2020-12-22 00:00:00+00:00,-34.208542,150.805571,609.6,-34.168488,150.822330,472.44
1581581,ZYU,NaN,7c8204,NaN,NaN,YWBN,YWBN,2020-12-26 20:52:15+00:00,2020-12-26 21:53:55+00:00,2020-12-26 00:00:00+00:00,-34.171484,150.808239,304.8,-34.167248,150.817394,419.10
1595248,ZYU,NaN,7c8204,NaN,NaN,YWBN,YWBN,2020-12-27 01:27:25+00:00,2020-12-27 01:51:03+00:00,2020-12-27 00:00:00+00:00,-34.169724,150.807701,304.8,-34.167892,150.815772,426.72


## ```covid_19``` table

In [7]:
s3_input_data = 's3a://udacity-capstoneproject/source/covid-19-world-cases-deaths-testing/dataset/covid-19-world-cases-deaths-testing.csv'
input_data = '../data/source/covid-19_world_cases_deaths_testing.csv'


covid19 = pd.read_csv( input_data)
covid19

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72091,ZWE,Africa,Zimbabwe,2021-02-25,35994.0,34.0,64.429,1458.0,2.0,5.429,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571
72092,ZWE,Africa,Zimbabwe,2021-02-26,36044.0,50.0,47.714,1463.0,5.0,4.714,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571
72093,ZWE,Africa,Zimbabwe,2021-02-27,36058.0,14.0,41.429,1463.0,0.0,4.429,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571
72094,ZWE,Africa,Zimbabwe,2021-02-28,36089.0,31.0,41.857,1463.0,0.0,3.857,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571


In [8]:
print(covid19.iso_code.unique())
print("Total number of iso_code: ", len(covid19.iso_code.unique()))

['AFG' 'OWID_AFR' 'ALB' 'DZA' 'AND' 'AGO' 'AIA' 'ATG' 'ARG' 'ARM'
 'OWID_ASI' 'AUS' 'AUT' 'AZE' 'BHS' 'BHR' 'BGD' 'BRB' 'BLR' 'BEL' 'BLZ'
 'BEN' 'BMU' 'BTN' 'BOL' 'BIH' 'BWA' 'BRA' 'BRN' 'BGR' 'BFA' 'BDI' 'KHM'
 'CMR' 'CAN' 'CPV' 'CYM' 'CAF' 'TCD' 'CHL' 'CHN' 'COL' 'COM' 'COG' 'CRI'
 'CIV' 'HRV' 'CUB' 'CYP' 'CZE' 'COD' 'DNK' 'DJI' 'DMA' 'DOM' 'ECU' 'EGY'
 'SLV' 'GNQ' 'ERI' 'EST' 'SWZ' 'ETH' 'OWID_EUR' 'OWID_EUN' 'FRO' 'FLK'
 'FJI' 'FIN' 'FRA' 'GAB' 'GMB' 'GEO' 'DEU' 'GHA' 'GIB' 'GRC' 'GRL' 'GRD'
 'GTM' 'GGY' 'GIN' 'GNB' 'GUY' 'HTI' 'HND' 'HKG' 'HUN' 'ISL' 'IND' 'IDN'
 'OWID_INT' 'IRN' 'IRQ' 'IRL' 'IMN' 'ISR' 'ITA' 'JAM' 'JPN' 'JEY' 'JOR'
 'KAZ' 'KEN' 'OWID_KOS' 'KWT' 'KGZ' 'LAO' 'LVA' 'LBN' 'LSO' 'LBR' 'LBY'
 'LIE' 'LTU' 'LUX' 'MAC' 'MDG' 'MWI' 'MYS' 'MDV' 'MLI' 'MLT' 'MHL' 'MRT'
 'MUS' 'MEX' 'FSM' 'MDA' 'MCO' 'MNG' 'MNE' 'MAR' 'MOZ' 'MMR' 'NAM' 'NPL'
 'NLD' 'NZL' 'NIC' 'NER' 'NGA' 'OWID_NAM' 'MKD' 'OWID_NCY' 'NOR'
 'OWID_OCE' 'OMN' 'PAK' 'PSE' 'PAN' 'PNG' 'PRY' 'PER' 'PHL' 'POL' 'PRT'

In [9]:
countries = pd.read_csv('../data/source/countries.csv')
print(countries.code.unique())
print("Total number of code in countries.csv: ", len(countries.code.unique()))

['AD' 'AE' 'AF' 'AG' 'AI' 'AL' 'AM' 'AO' 'AQ' 'AR' 'AS' 'AT' 'AU' 'AW'
 'AZ' 'BA' 'BB' 'BD' 'BE' 'BF' 'BG' 'BH' 'BI' 'BJ' 'BL' 'BM' 'BN' 'BO'
 'BQ' 'BR' 'BS' 'BT' 'BW' 'BY' 'BZ' 'CA' 'CC' 'CD' 'CF' 'CG' 'CH' 'CI'
 'CK' 'CL' 'CM' 'CN' 'CO' 'CR' 'CU' 'CV' 'CW' 'CX' 'CY' 'CZ' 'DE' 'DJ'
 'DK' 'DM' 'DO' 'DZ' 'EC' 'EE' 'EG' 'EH' 'ER' 'ES' 'ET' 'FI' 'FJ' 'FK'
 'FM' 'FO' 'FR' 'GA' 'GB' 'GD' 'GE' 'GF' 'GG' 'GH' 'GI' 'GL' 'GM' 'GN'
 'GP' 'GQ' 'GR' 'GS' 'GT' 'GU' 'GW' 'GY' 'HK' 'HN' 'HR' 'HT' 'HU' 'ID'
 'IE' 'IL' 'IM' 'IN' 'IO' 'IQ' 'IR' 'IS' 'IT' 'JE' 'JM' 'JO' 'JP' 'KE'
 'KG' 'KH' 'KI' 'KM' 'KN' 'KP' 'KR' 'KW' 'KY' 'KZ' 'LA' 'LB' 'LC' 'LI'
 'LK' 'LR' 'LS' 'LT' 'LU' 'LV' 'LY' 'MA' 'MC' 'MD' 'ME' 'MF' 'MG' 'MH'
 'MK' 'ML' 'MM' 'MN' 'MO' 'MP' 'MQ' 'MR' 'MS' 'MT' 'MU' 'MV' 'MW' 'MX'
 'MY' 'MZ' nan 'NC' 'NE' 'NF' 'NG' 'NI' 'NL' 'NO' 'NP' 'NR' 'NU' 'NZ' 'OM'
 'PA' 'PE' 'PF' 'PG' 'PH' 'PK' 'PL' 'PM' 'PN' 'PR' 'PS' 'PT' 'PW' 'PY'
 'QA' 'RE' 'RO' 'RS' 'RU' 'RW' 'SA' 'SB' 'SC' 'SD' 'SE' 'SG' 'SH' 'SI'
 '

In [10]:
print(covid19.location.unique())
print("Total number of locations: ", len(covid19.location.unique()))

['Afghanistan' 'Africa' 'Albania' 'Algeria' 'Andorra' 'Angola' 'Anguilla'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Asia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bermuda' 'Bhutan' 'Bolivia'
 'Bosnia and Herzegovina' 'Botswana' 'Brazil' 'Brunei' 'Bulgaria'
 'Burkina Faso' 'Burundi' 'Cambodia' 'Cameroon' 'Canada' 'Cape Verde'
 'Cayman Islands' 'Central African Republic' 'Chad' 'Chile' 'China'
 'Colombia' 'Comoros' 'Congo' 'Costa Rica' "Cote d'Ivoire" 'Croatia'
 'Cuba' 'Cyprus' 'Czechia' 'Democratic Republic of Congo' 'Denmark'
 'Djibouti' 'Dominica' 'Dominican Republic' 'Ecuador' 'Egypt'
 'El Salvador' 'Equatorial Guinea' 'Eritrea' 'Estonia' 'Eswatini'
 'Ethiopia' 'Europe' 'European Union' 'Faeroe Islands' 'Falkland Islands'
 'Fiji' 'Finland' 'France' 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana'
 'Gibraltar' 'Greece' 'Greenland' 'Grenada' 'Guatemala' 'Guernsey'
 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti' 'H

In [11]:
covid19_countries = covid19[['iso_code', 'continent', 'location']].drop_duplicates()
covid19_countries

,iso_code,continent,location
0,AFG,Asia,Afghanistan
372,OWID_AFR,NaN,Africa
755,ALB,Europe,Albania
1126,DZA,Africa,Algeria
1497,AND,Europe,Andorra
...,...,...,...
70265,VNM,Asia,Vietnam
70669,OWID_WRL,NaN,World
71074,YEM,Asia,Yemen
71400,ZMB,Africa,Zambia


In [12]:
covid19_countries.loc[covid19_countries.iso_code.str.contains('OWID')]

,iso_code,continent,location
372,OWID_AFR,NaN,Africa
3366,OWID_ASI,NaN,Asia
21986,OWID_EUR,NaN,Europe
22390,OWID_EUN,NaN,European Union
30370,OWID_INT,NaN,International
34556,OWID_KOS,Europe,Kosovo
47376,OWID_NAM,NaN,North America
48151,OWID_NCY,NaN,Northern Cyprus
48533,OWID_OCE,NaN,Oceania
59766,OWID_SAM,NaN,South America


In [13]:
joined_countries = countries.join( covid19_countries.set_index('location'),
               on='name', how='left',
               lsuffix='_cts', rsuffix='_cvd')
joined_countries

,id,code,name,continent_cts,wikipedia_link,keywords,iso_code,continent_cvd
0,302672,AD,Andorra,EU,https://en.wikipedia.org/wiki/Andorra,NaN,AND,Europe
1,302618,AE,United Arab Emirates,AS,https://en.wikipedia.org/wiki/United_Arab_Emir...,"UAE,مطارات في الإمارات العربية المتحدة",ARE,Asia
2,302619,AF,Afghanistan,AS,https://en.wikipedia.org/wiki/Afghanistan,NaN,AFG,Asia
3,302722,AG,Antigua and Barbuda,NaN,https://en.wikipedia.org/wiki/Antigua_and_Barbuda,NaN,ATG,North America
4,302723,AI,Anguilla,NaN,https://en.wikipedia.org/wiki/Anguilla,NaN,AIA,North America
...,...,...,...,...,...,...,...,...
242,302609,YT,Mayotte,AF,https://en.wikipedia.org/wiki/Mayotte,NaN,NaN,NaN
243,302610,ZA,South Africa,AF,https://en.wikipedia.org/wiki/South_Africa,NaN,ZAF,Africa
244,302611,ZM,Zambia,AF,https://en.wikipedia.org/wiki/Zambia,NaN,ZMB,Africa
245,302612,ZW,Zimbabwe,AF,https://en.wikipedia.org/wiki/Zimbabwe,NaN,ZWE,Africa


In [14]:
joined_countries.isna().sum()

id                  0
code                1
name                0
continent_cts      41
wikipedia_link      0
keywords          155
iso_code           56
continent_cvd      56
dtype: int64

In [15]:
joined_countries.loc[ joined_countries.code.isna()]

,id,code,name,continent_cts,wikipedia_link,keywords,iso_code,continent_cvd
156,302591,NaN,Namibia,AF,https://en.wikipedia.org/wiki/Namibia,NaN,NAM,Africa


In [16]:
cases = covid19[['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million']]
cases.isna().sum()

iso_code                               0
continent                           3580
location                               0
date                                   0
total_cases                          964
new_cases                            966
new_cases_smoothed                  1967
total_deaths                       10054
new_deaths                          9896
new_deaths_smoothed                 1967
total_cases_per_million             1353
new_cases_per_million               1355
new_cases_smoothed_per_million      2351
total_deaths_per_million           10430
new_deaths_per_million             10272
new_deaths_smoothed_per_million     2351
dtype: int64